In [19]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("ADS project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

In [20]:
import pandas as pd
import numpy as np

raw_data = "../data/ABS/"
curated_data = "../data/curated/"

tbl_income = pd.read_excel(raw_data+"annual income by SA2 regions.xlsx")
tbl_population = pd.read_excel(raw_data+"population estimates by SA2 regions.xlsx")


In [21]:
tbl_income.drop(np.arange(0,8), inplace=True)
tbl_income.drop(np.arange(2303,2306), inplace=True)

tbl_income.drop(tbl_income.columns[[2,3,4,5,7,8,9,10,12,13,14,15,17,18,19,20,22,23,24,25,27]], axis=1, inplace=True)
tbl_income = tbl_income.reset_index(drop=True)
tbl_income.columns = ['sa2', 'region', 'total_earners', 'median_age', 'income_sum', 'income_median', 'income_mean']
tbl_income


,sa2,region,total_earners,median_age,income_sum,income_median,income_mean
0,101021007,Braidwood,2361,51,120763285,41593,51149
1,101021008,Karabar,5100,42,338308979,61777,66335
2,101021009,Queanbeyan,6697,39,441160946,60119,65874
3,101021010,Queanbeyan - East,3393,40,237035656,63051,69860
4,101021011,Queanbeyan Region,12821,44,1050285692,71420,81919
...,...,...,...,...,...,...,...
2290,801101137,Molonglo,np,np,np,np,np
2291,801101138,Molonglo - North,np,np,np,np,np
2292,801101139,Wright,2121,35,182421537,79150,86007
2293,801111140,ACT - South West,355,40,26069449,64227,73435


In [22]:
tbl_income[tbl_income["sa2"]==101021007]

,sa2,region,total_earners,median_age,income_sum,income_median,income_mean
0,101021007,Braidwood,2361,51,120763285,41593,51149


In [23]:
tbl_income.to_csv(curated_data+"clean_annual.csv")

In [24]:
tbl_population.drop(tbl_population.columns[0:8], axis=1, inplace=True)
tbl_population.drop(tbl_population.columns[1:22], axis=1, inplace=True)
tbl_population.drop(tbl_population.columns[2:6], axis=1, inplace=True)
tbl_population

,SA2 code,2021,km2,persons/km2
0,101021007.0,4330.0,3418.4,1.3
1,101021008.0,8546.0,7.0,1223.9
2,101021009.0,11370.0,4.8,2387.7
3,101021010.0,5093.0,13.0,391.7
4,101021012.0,12743.0,13.7,931.9
...,...,...,...,...
2456,NaN,NaN,NaN,NaN
2457,NaN,NaN,NaN,NaN
2458,NaN,NaN,NaN,NaN
2459,NaN,NaN,NaN,NaN


In [25]:
tbl_population.columns = ["sa2", "2021", "km2", "persons/km2"]
tbl_population

,sa2,2021,km2,persons/km2
0,101021007.0,4330.0,3418.4,1.3
1,101021008.0,8546.0,7.0,1223.9
2,101021009.0,11370.0,4.8,2387.7
3,101021010.0,5093.0,13.0,391.7
4,101021012.0,12743.0,13.7,931.9
...,...,...,...,...
2456,NaN,NaN,NaN,NaN
2457,NaN,NaN,NaN,NaN
2458,NaN,NaN,NaN,NaN
2459,NaN,NaN,NaN,NaN


In [26]:
tbl_population.to_csv(curated_data+"clean_population.csv")

In [27]:
postcode_2011_SA2_2011 = pd.ExcelFile(raw_data+"1270055006_CG_POSTCODE_2011_SA2_2011.xls")
postcode_sa2 = pd.read_excel(postcode_2011_SA2_2011, "Table 3")

In [28]:
postcode_sa2.drop(np.arange(0,6), inplace=True)
postcode_sa2.drop(np.arange(5994,5997), inplace=True)

postcode_sa2.drop(postcode_sa2.columns[3:6], axis=1, inplace=True)
postcode_sa2.drop(postcode_sa2.columns[0], axis=1, inplace=True)
postcode_sa2 = postcode_sa2.reset_index(drop=True)

postcode_sa2.columns = ['postcode', 'sa2']
postcode_sa2['postcode'] = pd.to_numeric(postcode_sa2['postcode'])
postcode_sa2['sa2'] = pd.to_numeric(postcode_sa2['sa2'])

In [29]:
postcode_sa2

,postcode,sa2
0,800,701011002
1,810,701021010
2,810,701021013
3,810,701021016
4,810,701021018
...,...,...
5983,7466,604031097
5984,7467,604031097
5985,7468,604031097
5986,7469,604031097


In [30]:
postcode_sa2.to_csv(curated_data+"postcode_sa2.csv")

In [31]:
tbl_income = tbl_income.merge(postcode_sa2, on='sa2', how='left')
tbl_income.drop(tbl_income.columns[-2], axis=1, inplace=True)

In [32]:
tbl_income = tbl_income.merge(tbl_population, on='sa2', how='left')

In [39]:
import numpy as np
tbl_income = tbl_income[tbl_income["postcode"] != np.nan]
# tbl_income['postcode'] = tbl_income['postcode'].astype(str)

In [40]:
tbl_income

,sa2,region,total_earners,median_age,income_sum,income_median,postcode,2021,km2,persons/km2


In [35]:
mechant_consumer_info = pd.read_parquet(curated_data+"mechant_consumer_info")

In [36]:
mechant_consumer_info

,merchant_name,products,revenue_level,take_rate,dollar_value,order_year,order_month,order_day,consumer,consumer_address,consumer_state,consumer_postcode,consumer_gender
0,Nullam Enim Sed Incorporated,"[tent, and, awning, shops]",e,0.27,24.224445,2021,7,29,Todd Long,883 Patty Mountains Apt. 285,NSW,2021,Male
1,Felis Limited,"[furniture, home, furnishings, and, equipment,...",e,0.18,212.857318,2021,5,2,Todd Long,883 Patty Mountains Apt. 285,NSW,2021,Male
2,Non Lobortis Corporation,"[computer, programming, data, processing, and,...",e,0.36,168.526052,2021,3,11,Anthony Bryant,76494 Derrick Court,VIC,3874,Male
3,Eget Metus Eu Institute,"[shoe, shops]",e,0.42,328.258033,2021,7,2,Christopher Rodriguez,30554 Evans Stream Apt. 379,NSW,2299,Male
4,Et Nunc Consulting,"[books, periodicals, and, newspapers]",e,0.16,979.978796,2021,8,15,Sara Allen,92887 Stewart Flat,NSW,2798,Undisclosed
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6988071,Dictum Ultricies Ltd,"[computer, programming, data, processing, and,...",a,6.43,570.485193,2021,6,19,Michelle Matthews,2887 Joanne Vista,VIC,3201,Female
6988072,Ornare Fusce Inc.,"[hobby, toy, and, game, shops]",a,5.98,189.266530,2021,7,20,Michelle Matthews,2887 Joanne Vista,VIC,3201,Female
6988073,Orci In Consequat Corporation,"[gift, card, novelty, and, souvenir, shops]",a,6.61,77.808667,2021,7,3,Michelle Matthews,2887 Joanne Vista,VIC,3201,Female
6988074,Vel Est Tempor LLP,"[computers, computer, peripheral, equipment, a...",a,5.76,10.768560,2021,6,28,Michelle Matthews,2887 Joanne Vista,VIC,3201,Female
